In [17]:
import os
import sqlalchemy.orm.decl_api
import time

from api import *
from dataframe import *
from database import *
from file import create_outfile
from unaccent import unaccent

# cfg

In [18]:
cfg = {
    "user": f"{os.environ['POSTGRE_USER']}",
    "password": f"{os.environ['POSTGRE_PASSWORD']}",
    "port": "5432",
    "database": "herbario"
}
filename = "original.csv"
filename_george = "dados-george.csv"

engine, session = connect(cfg)

problems with host localhost
2022-07-22 14:32:48,765 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-07-22 14:32:48,781 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-22 14:32:48,781 INFO sqlalchemy.engine.Engine select current_schema()
2022-07-22 14:32:48,781 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-22 14:32:48,781 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-07-22 14:32:48,781 INFO sqlalchemy.engine.Engine [raw sql] {}


# Load data

In [19]:
dataframe = pandas.read_csv("original.csv", sep=";", low_memory=False, skipinitialspace=True)
dataframe_george = pandas.read_csv(filename_george, sep=";", low_memory=False, skipinitialspace=True)

In [22]:
create_table_if_not_exists(cfg, engine, "county")
create_table_if_not_exists(cfg, engine, "data")

2022-07-22 14:32:49,394 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2022-07-22 14:32:49,394 INFO sqlalchemy.engine.Engine [generated in 0.00106s] {'schema': 'herbario', 'name': 'county'}
2022-07-22 14:32:49,394 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-22 14:32:49,410 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-07-22 14:32:49,410 INFO sqlalchemy.engine.Engine [generated in 0.00092s] {'name': 'data'}
2022-07-22 14:32:49,410 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-07-22 14:32:49,410 INFO sqlalchemy.engine.Engine [cached since 0.006581s ago] {'name': 'county'}
2022-07-22 14:32:49,410 INFO sqlalchemy.engine.E

In [23]:
if session.query(County).count() == 0:
    response = get_municipies()
    for i, county in enumerate(response.json()):
        session.add(create_county(county))
        make_operation(session)


if session.query(DataSP).count() == 0:
    dataframe = preprocess(dataframe)
    for row in dataframe.iterrows():
        session.add(create_datasp(row[1]))
        make_operation(session)


if session.query(DataSP).filter(DataSP.george == True).count() == 0:
    for row in dataframe.iterrows():
        if row[1]["GEORGE"].lower() == "sim":
            session.query(DataSP).filter(DataSP.seq == row[1]["seq"]).update({"george": True}, synchronize_session=False)
            make_operation(session)

2022-07-22 14:32:49,457 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-22 14:32:49,457 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT county.id AS county_id, county.county AS county_county, county.uf AS county_uf, county.uf_name AS county_uf_name 
FROM county) AS anon_1
2022-07-22 14:32:49,457 INFO sqlalchemy.engine.Engine [generated in 0.00105s] {}
2022-07-22 14:32:49,472 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT data.seq AS data_seq, data.modified AS data_modified, data.institution_code AS data_institution_code, data.collection_code AS data_collection_code, data.catalog_number AS data_catalog_number, data.basis_of_record AS data_basis_of_record, data.kingdom AS data_kingdom, data.phylum AS data_phylum, data.classe AS data_classe, data."order" AS data_order, data.family AS data_family, data.genus AS data_genus, data.specific_epithet AS data_specific_epithet, data.infraspecific_epithet AS data_infraspecific_epithet, data.scie

In [24]:
def column_is_string_or_varchar_or_text(column):
    return str(column.type).lower() in ("string", "varchar", "text")

def get_columns_text():
    return list([column.key for column in get_columns_table(DataSP) if column_is_string_or_varchar_or_text(column)])


def remove_white_spaces(string):
    return remove_word_started_lowercase(string).split(" ")


def remove_hyphen(string):
    return remove_word_started_lowercase(string).split("-")


def remove_word_started_lowercase(string):
    return re.sub(r"\b[a-z]+\s*", "", string)


def find_and_update(data, list_seq, new_column, value=None):
    for columns in list_of_columns_valid:
        for data_formatted in (remove_white_spaces(data), remove_hyphen(data)):
            if len(data_formatted) > 0:
                q = session.query(DataSP.seq).filter(sqlalchemy.and_(
                    *[sqlalchemy.func.lower(unaccent(getattr(DataSP, columns))).ilike(f"%{s.lower()}%") for s in
                      data_formatted])).all()
            else:
                q = session.query(DataSP.seq).filter(
                    sqlalchemy.func.lower(unaccent(getattr(DataSP, columns))).ilike(f"%{d}%")).all()
            if len(q) > 0:
                list_seq.append({"seq": remove_set(q), "column": columns, "new_column": new_column, "value_searched": value if value else data})

In [25]:
data_piperaceae = session.query(DataSP).all()
data_uf = session.query(sqlalchemy.func.lower(unaccent(County.uf))).distinct().all()
data_uf_name = session.query(sqlalchemy.func.lower(unaccent(County.uf_name))).distinct().all()
data_county = session.query(unaccent(County.county)).all()

2022-07-22 14:32:49,542 INFO sqlalchemy.engine.Engine SELECT data.seq AS data_seq, data.modified AS data_modified, data.institution_code AS data_institution_code, data.collection_code AS data_collection_code, data.catalog_number AS data_catalog_number, data.basis_of_record AS data_basis_of_record, data.kingdom AS data_kingdom, data.phylum AS data_phylum, data.classe AS data_classe, data."order" AS data_order, data.family AS data_family, data.genus AS data_genus, data.specific_epithet AS data_specific_epithet, data.infraspecific_epithet AS data_infraspecific_epithet, data.scientific_name AS data_scientific_name, data.scientific_name_authorship AS data_scientific_name_authorship, data.identified_by AS data_identified_by, data.year_identified AS data_year_identified, data.month_identified AS data_month_identified, data.day_identified AS data_day_identified, data.type_status AS data_type_status, data.recorded_by AS data_recorded_by, data.record_number AS data_record_number, data.field_numb

In [26]:
def remove_set(data):
    return list([s for s, in data])

data_uf = remove_set(data_uf)
data_uf_name = remove_set(data_uf_name)
data_county = remove_set(data_county)
data_country = list(["Brazil", "Brasil"])

In [28]:
list_seq = list([])
list_of_columns_valid = get_columns_text()

In [ ]:
start = time.process_time()

for d in data_country:
    find_and_update(d, list_seq, "my_country", value="Brasil")

end = time.process_time()
elapsed_time = end-start
print(f"elapsed_time: {elapsed_time} seconds")

In [ ]:
start = time.process_time()

for d in data_uf_name:
    find_and_update(d, list_seq, "my_state")

end = time.process_time()
elapsed_time = end-start
print(f"elapsed_time: {elapsed_time} seconds")

In [ ]:
start = time.process_time()

for d in data_county:
    find_and_update(d, list_seq, "my_city")

end = time.process_time()
elapsed_time = end-start
print(f"elapsed_time: {elapsed_time} seconds")

In [ ]:
for l in list_seq:
    for y in l["seq"]:
        session.query(DataSP).filter(DataSP.seq == y).update(values=dict({l["new_column"]: l["value_searched"]}), synchronize_session=False)
        make_operation(session)

In [ ]:
create_outfile(list_seq)

In [ ]:
session.close()
engine.dispose()